In [ ]:
!pip install pymorphy2

In [ ]:
import pylab
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)

In [ ]:
import nltk
nltk.download()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


-----

# Задание 1

1. С помощью *pd.read_csv()* загрузите датафреймы positive.csv и negative.csv (обратите внимание, что исходные таблицы не содержат наименования столбцов и на первой строке располагаются данные. Файлы расположены в папке datasets);
2. Объедините датафреймы с помощью *pd.concat()* в один датафрейм;
3. Убедитесь, что в новом датафрейме индексация сквозная и без повторов;
4. Переименуйте столбцы датафрейма (столбцы полностью соответствуют примеру);
5. Выведите информацию об общем количестве полученных твитов, сколько из них негативных, сколько позитивных.

-----

Инициализируем 2 датафрейма:
df_pos - датафрейм с позитивными комментариями
df_neg - датафрейм с негативными комментариями

In [ ]:
df_pos = pd.read_csv('/content/drive/MyDrive/positive.csv', sep=';', header=None)
df_neg = pd.read_csv('/content/drive/MyDrive/negative.csv', sep=';', header=None)

Объединяем 2 датафрейма в 1 общий - df_tweets, после чего обнулим предыдущие

In [ ]:
df_tweets = pd.concat([df_pos, df_neg], ignore_index=True)
df_pos = pd.DataFrame(None)
df_neg = pd.DataFrame(None)
df_tweets

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [ ]:
df_tweets.columns = ['id', 'date', 'name', 'text', 'positive',
                     'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'list_count']
df_tweets

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


Выведем информацию об общем количестве полученных твитов, сколько из них негативных, сколько позитивных

In [ ]:
print(f'Общее количество твитов - {len(df_tweets.id)}')
print(f'Из них негативных - {df_tweets["positive"].value_counts()[1]}')
print(f'Позитивных - {df_tweets["positive"].value_counts()[-1]}')

Общее количество твитов - 226834
Из них негативных - 114911
Позитивных - 111923


----

# Задание 2

Произведите очистку данных, сформированных в задании 1. По результатам очистки выведите на экран следующую информацию:
- Общее количество слов перед удалением слов, встречающихся 1 раз;
- Количество слов, встречающихся 1 раз;
- Итоговое количество слов;
- Количество пустых твитов;
- Из них позитивных твитов;
- Количество твитов после удаления пустых.

----

Приведем все слова к строчным буквам и оставим в тексте только русские слова, удалив числа, знаки препинания, специальные символы и слова написанные латиницей:


In [ ]:
df_tweets.text = df_tweets.text.str.lower()
df_tweets.text = df_tweets.text.str.replace(r"[^А-Яа-я]"," ")
df_tweets

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,хоть я и школота но поверь у на...,1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,да все таки он немного похож на него но мой ...,1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,ну ты идиотка я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,кто то в углу сидит и погибае...,1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,вот что значит страшилка но...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,но не каждый хочет что то исправлять ...,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучаю так только вправляет мо...,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,вот и в школу в говно это идти уже надо,-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,тауриэль не грусти ...,-1,0,1,0,2516,187,265,0


Разобьем тексты на слова с помощью word_tokenize:

In [ ]:
from nltk.tokenize import word_tokenize
df_tweets.text = list(map(word_tokenize, df_tweets.text))

In [ ]:
df_tweets

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"[хоть, я, и, школота, но, поверь, у, нас, то, ...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"[да, все, таки, он, немного, похож, на, него, ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,"[ну, ты, идиотка, я, испугалась, за, тебя]",1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"[кто, то, в, углу, сидит, и, погибает, от, гол...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,"[вот, что, значит, страшилка, но, блин, посмот...",1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,"[но, не, каждый, хочет, что, то, исправлять]",-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,"[скучаю, так, только, вправляет, мозги, но, я,...",-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"[вот, и, в, школу, в, говно, это, идти, уже, н...",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"[тауриэль, не, грусти, обнял]",-1,0,1,0,2516,187,265,0


Удалим стоп-слова

In [ ]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()


def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df_tweets.text = list(map(delete_stopword, df_tweets.text))

In [ ]:
df_tweets

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"[школота, поверь, самое, общество, профилирующ...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"[таки, немного, похож, мальчик, равно]",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,"[идиотка, испугалась]",1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"[углу, сидит, погибает, голода, ещ, порции, вз...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,"[значит, страшилка, блин, посмотрев, части, со...",1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,"[каждый, хочет, исправлять]",-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,"[скучаю, вправляет, мозги, равно, скучаю]",-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"[школу, говно, это, идти]",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"[тауриэль, грусти, обнял]",-1,0,1,0,2516,187,265,0


Проведем лемматизацию полученных слов

In [ ]:
%%time
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s


df_tweets.text = list(map(lemmatization, df_tweets.text))

CPU times: user 5min 4s, sys: 1.23 s, total: 5min 5s
Wall time: 5min 6s


Далее необходимо удалить все слова, которые встречаются только 1 раз
Но перед этим посчитаем общее количество слов и количество слов и количество слов, которые встречаются только 1 раз

In [ ]:
from nltk.probability import FreqDist

def to_str(s):
    new_s = ' '.join(j for j in s)
    return new_s

def delete_word(words):
    global words_to_del
    new_s = [word for word in words if word not in words_to_del]
    return new_s


text_tokens = word_tokenize(' '.join(j for j in list(map(to_str, df_tweets.text))))
text = nltk.Text(text_tokens)
fdist = FreqDist(text)
words_to_del = list(filter(lambda k: fdist[k] == 1, fdist))
print(f'Общее количество слов - {len(text_tokens)}')
print(f'Количество слов, встречающихся только 1 раз - {len(words_to_del)}')

Общее количество слов - 1493275
Количество слов, встречающихся только 1 раз - 53568


Удалим эти слова

In [ ]:
words_to_del = set(words_to_del)

In [ ]:
%%time
df_tweets.text = list(map(delete_word, df_tweets.text))
df_tweets.text = list(map(to_str, df_tweets.text))

CPU times: user 2.78 s, sys: 15 ms, total: 2.8 s
Wall time: 2.92 s


Посчитаем количество пустых твитов после чистки, а также сколько из них положительных

In [ ]:
print(f'Количество пустых твитов - {len(df_tweets[df_tweets.text == ""])}')
print(f'Из них положительных - {len(df_tweets[(df_tweets.text == "") & (df_tweets.positive == 1)])}')

Количество пустых твитов - 856
Из них положительных - 382


Удалим все пустые твиты, после чего посчитаем, сколько твитов у нас осталось после очистки дата фрейма


In [ ]:
df_tweets = df_tweets.drop(df_tweets[df_tweets.text == ''].index, axis = 0)
print(f'Количество твитов после очистки данных - {len(df_tweets)}')

Количество твитов после очистки данных - 225978


---

# Задание 3

1. Кодировать данные методом мешка слов.
2. Кодировать данные методом TF-IDF.
3. Построить классификатор на основе логистической регрессии, используя мешок слов.
4. Построить классификатор на основе логистической регрессии, используя TF-IDF.
5. Построить классификатор на основе случайного леса, используя мешок слов.
6. Построить классификатор на основе случайного леса, используя TF-IDF.
7. Сделайте выводы о разработанных классификаторах.

---

При разбиении на обучающую и тестовую выборки, следует указать *test_size=0.3*

---

***Рекомендация:*** для случайного леса параметр n_estimator должен быть не менее 200

Перед выполнением задания 3 создам копию последней версии дата сета (на всякий случай буду работать с ним)

In [ ]:
df_tweets_copy = df_tweets

Разобьем дата сет на обучающую и тренировочную выборку

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_tweets_copy.text, df_tweets_copy.positive, test_size=0.3, random_state=21)

Кодируем наши данные методом мешка слов и методом TF-IDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
tfidf_test = tfidf.transform(X_test)

Построим классификатор на основе логической регрессии и кодировки методом мешка слов

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(cv_train, y_train)
cv_pred = lr.predict(cv_test)

print('Результаты классификации на тестовой выборке:')
print()
print(classification_report(y_test, cv_pred))

print('Результаты классификации на тренировочной выборке:')
print()
print(classification_report(y_train, lr.predict(cv_train)))

Результаты классификатора на тестовой выборке:

              precision    recall  f1-score   support

          -1       0.72      0.71      0.71     33422
           1       0.72      0.73      0.72     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

Результаты классификатора на тестовой выборке:

              precision    recall  f1-score   support

          -1       0.80      0.79      0.79     78027
           1       0.80      0.80      0.80     80157

    accuracy                           0.80    158184
   macro avg       0.80      0.80      0.80    158184
weighted avg       0.80      0.80      0.80    158184



Построим классификатор на основе логической регрессии и кодировки методом TF-IDF

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(tfidf_train, y_train)
tfidf_pred = lr.predict(tfidf_test)

print('Результаты классификации на тестовой выборке:')
print()
print(classification_report(y_test, tfidf_pred))

print('Результаты классификации на тренировочной выборке:')
print()
print(classification_report(y_train, lr.predict(tfidf_train)))

Результаты классификации на тестовой выборке:

              precision    recall  f1-score   support

          -1       0.73      0.69      0.71     33422
           1       0.71      0.75      0.73     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

Результаты классификации на тестовой выборке:

              precision    recall  f1-score   support

          -1       0.79      0.75      0.77     78027
           1       0.77      0.80      0.79     80157

    accuracy                           0.78    158184
   macro avg       0.78      0.78      0.78    158184
weighted avg       0.78      0.78      0.78    158184



Построим классификатор на основе случайного леса и кодировки методом мешка слов

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=250, n_jobs=-1, random_state=21)
forest.fit(cv_train, y_train)
cv_pred = forest.predict(cv_test)

print('Результаты классификации на тестовой выборке:')
print()
print(classification_report(y_test, cv_pred))

print('Результаты классификации на тренировочной выборке:')
print()
print(classification_report(y_train, forest.predict(cv_train)))

Результаты классификации на тестовой выборке:

              precision    recall  f1-score   support

          -1       0.68      0.76      0.72     33422
           1       0.74      0.65      0.69     34372

    accuracy                           0.70     67794
   macro avg       0.71      0.70      0.70     67794
weighted avg       0.71      0.70      0.70     67794

Результаты классификации на тренировочной выборке:

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00     78027
           1       1.00      1.00      1.00     80157

    accuracy                           1.00    158184
   macro avg       1.00      1.00      1.00    158184
weighted avg       1.00      1.00      1.00    158184

CPU times: total: 2h 8min 50s
Wall time: 16min 56s


Построим классификатор на основе случайного леса и кодировки методом TF-IDF

In [ ]:
%%time
forest.fit(tfidf_train, y_train)
tfidf_pred = forest.predict(tfidf_test)

print('Результаты классификации на тестовой выборке:')
print()
print(classification_report(y_test, tfidf_pred))

print('Результаты классификации на тренировочной выборке:')
print()
print(classification_report(y_train, forest.predict(tfidf_train)))

Результаты классификации на тестовой выборке:

              precision    recall  f1-score   support

          -1       0.72      0.70      0.71     33422
           1       0.72      0.74      0.73     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

Результаты классификации на тренировочной выборке:

              precision    recall  f1-score   support

          -1       0.99      1.00      1.00     78027
           1       1.00      1.00      1.00     80157

    accuracy                           1.00    158184
   macro avg       1.00      1.00      1.00    158184
weighted avg       1.00      1.00      1.00    158184

CPU times: total: 1h 57min 57s
Wall time: 15min 23s


Исхдя из полученных предсказаний можно сделать вывод, что наши классификаторы не то, чтобы с высокой точностью предсказывают тональность твита. Однако точность предсказания можно улучшить, если не удалять из твитов скобочки смайликов на этапе очистки данных.

Другой момент, который прослеживается на классификаторе случайного леса - это переобучение. На тренировочной выборке точность все время находится около 1, однако на тестовой точность резко падает.

Из-за большой ресурсоемкости, запустить классификатор с множеством различных значений n_estimators с целью выяснить, при каком гиперпараметре не будет происходить переобучения, не представляется возможным

Однако, лично у меня при нескольких различных тестах, прослеживалось следующее:
При n_estimators=400 точность на тестовой выборке оставалась в пределах 1, но на тестовой резко падала (примерно до 0.64)
При n_estimators=200 наблюдалась такая же картина

А при n_estimators=250 точность для тестовой выборки была наилучшей